In [1]:
import os
import sys
import time
import tensorflow as tf

os.chdir('/scratch/edk202/word2gm-fast/notebooks')
os.chdir("..")

from src.word2gm_fast.dataprep.corpus_to_dataset import make_dataset
from src.word2gm_fast.dataprep.index_vocab import make_vocab
from src.word2gm_fast.dataprep.dataset_to_triplets import build_skipgram_triplets

2025-06-21 01:26:45.295294: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-21 01:26:45.311914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750483605.329218 2870105 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750483605.334406 2870105 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750483605.348141 2870105 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
!python -m unittest -b tests.test_corpus_to_dataset

2025-06-21 00:40:52.893344: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-21 00:40:52.907089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750480852.923017 3902574 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750480852.927811 3902574 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750480852.940672 3902574 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
!python -m unittest -b tests.test_index_vocab

2025-06-21 00:40:56.420920: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-21 00:40:56.434513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750480856.450534 3902645 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750480856.455334 3902645 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750480856.468235 3902645 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
!python -m unittest -v tests.test_dataset_to_triplets

2025-06-21 00:40:59.807210: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-21 00:40:59.820766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750480859.836531 3902688 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750480859.841335 3902688 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750480859.854048 3902688 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Corpus file information
corpus_file = "1850.txt"
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"
corpus_path = os.path.join(corpus_dir, corpus_file)
file_size = os.path.getsize(corpus_path) / 1024 / 1024
print("CORPUS FILE: ", corpus_path, "\n")

# Load and filter the corpus
start = time.time()
dataset, _ = make_dataset(corpus_path)
duration_load = time.time() - start
rate_load = file_size / duration_load
dataset = dataset.cache()

# Build the vocabulary from the dataset
start = time.time()
vocab_table = make_vocab(dataset)
duration_vocab = time.time() - start
rate_vocab = file_size / duration_vocab

# Make triplets from the dataset and vocabulary table
start = time.time()
triplets_ds = build_skipgram_triplets(dataset, vocab_table)
duration_triplets = time.time() - start
rate_triplets = file_size / duration_triplets

# Benchmarking output
print("[--    Benchmarks    --]\n")
print(f"{'Step':<35}{'Duration':>10}{'Quantity':>21}{'Rate':>21}")
print("-" * 87)
print(f"{'Corpus loading and filtering (Lazy)':<35}{duration_load:8,.2f}{'s':>2}{file_size:18,.2f}{'MB':>3}{rate_load:16,.2f}{'MB/s':>5}")
print(f"{'Vocabulary creation':<35}{duration_vocab:8,.2f}{'s':>2}{file_size:18,.2f}{'MB':>3}{rate_vocab:16,.2f}{'MB/s':>5}")
print(f"{'Triplet generation (Lazy)':<35}{duration_triplets:8,.2f}{'s':>2}{file_size:18,.2f}{'MB':>3}{rate_triplets:16,.2f}{'MB/s':>5}")

# Create reverse lookup from vocab table once at the beginning
vocab_export = vocab_table.export()
vocab_keys = vocab_export[0].numpy()
vocab_values = vocab_export[1].numpy()
index_to_word = {idx: word.decode('utf-8') for word, idx in zip(vocab_keys, vocab_values)}

# Show sample lines (use a fresh iterator)
print("\n[--   Sample Lines   --]\n")
sample_lines = list(dataset.shuffle(1000, seed=42).take(5).as_numpy_iterator())
for line_bytes in sample_lines:
    print(line_bytes.decode("utf-8"))

# Test the vocab table with example words
print("\n[--    Test Words    --]\n")
test_words = ["UNK", "man", "king", "nonexistentword"]
ids = vocab_table.lookup(tf.constant(test_words)).numpy()
print(f"{'Word':<18} {'ID':>6}")
print("-" * 25)
for word, idx in zip(test_words, ids):
    print(f"{word:<18} {idx:>6}")

# Show sample triplets
print("\n[--  Sample Triplets  --]\n")
print(f"{'Center':<8} {'Center Word':<12} {'Positive':<8} {'Pos Word':<12} {'Negative':<8} {'Neg Word':<12}")
print("-" * 75)

# Get sample triplets (use a fresh iterator with seed for reproducibility)
sample_triplets = list(triplets_ds.shuffle(1000, seed=123).take(5).as_numpy_iterator())
for triplet in sample_triplets:
    center, positive, negative = triplet
    center_word = index_to_word.get(center, f"ID_{center}")
    pos_word = index_to_word.get(positive, f"ID_{positive}")
    neg_word = index_to_word.get(negative, f"ID_{negative}")
    print(f"{center:<8} {center_word:<12} {positive:<8} {pos_word:<12} {negative:<8} {neg_word:<12}")


CORPUS FILE:  /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1850.txt 



2025-06-21 01:26:50.080213: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-06-21 01:28:54.910835: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-06-21 01:28:54.910835: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


[--    Benchmarks    --]

Step                                 Duration             Quantity                 Rate
---------------------------------------------------------------------------------------
Corpus loading and filtering           0.29 s            120.81 MB          419.37 MB/s
Vocabulary creation                  124.59 s            120.81 MB            0.97 MB/s
Triplet generation                     0.15 s            120.81 MB          788.08 MB/s

[--   Sample Lines   --]



2025-06-21 01:28:55.449700: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


UNK one another UNK two
UNK one day UNK serve
UNK one another UNK good
UNK one afore long UNK
UNK one back UNK UNK

[--    Test Words    --]

Word                   ID
-------------------------
UNK                     0
man                 18349
king                16702
nonexistentword         0

[--  Sample Triplets  --]

Center   Center Word  Positive Pos Word     Negative Neg Word    
---------------------------------------------------------------------------
1638     article      24205    rank         6700     corrigan    
2515     battle       20956    one          13066    gratefully  
3239     blood        20956    one          4585     carpentaria 
833      almost       15475    infinite     10444    evangelicals
1658     artorius     982      among        14982    impending   
1638     article      24205    rank         6700     corrigan    
2515     battle       20956    one          13066    gratefully  
3239     blood        20956    one          4585     carpentaria 
833 

In [ ]:
# Import TFRecord utilities
from src.word2gm_fast.dataprep.tfrecord_io import (
    save_pipeline_artifacts,
    load_pipeline_artifacts,
    write_vocab_to_tfrecord,
    load_vocab_from_tfrecord
)

In [ ]:
# Demonstrate TFRecord functionality (optional - commented out for speed)
print("\n[-- TFRecord Demo (Fast Pipeline Serialization) --]\n")

# Example usage (uncomment to actually save/load):
output_dir = "./pipeline_tfrecords"

# # Save all pipeline artifacts to TFRecords (one-time cost)
# print("Saving pipeline artifacts...")
# artifacts = save_pipeline_artifacts(
#     dataset, vocab_table, triplets_ds, 
#     output_dir=output_dir, 
#     compress=True
# )
# print(f"Saved to: {artifacts['output_dir']}")

# # Load pipeline artifacts (fast for training)
# print("\nLoading pipeline artifacts...")
# loaded = load_pipeline_artifacts(output_dir)
# loaded_vocab = loaded['vocab_table']
# loaded_triplets = loaded['triplets_ds']
# print(f"Loaded vocab size: {loaded['vocab_size']:,}")

print("TFRecord workflow allows:")
print("  1. One-time preprocessing: corpus → vocab + triplets → TFRecords")
print("  2. Fast training setup: TFRecords → ready-to-train datasets")
print("  3. Reproducible experiments with identical vocab/data")
print("  4. Easy sharing of preprocessed datasets")
print("\nFor large corpora, this eliminates 15+ minute preprocessing on each run!")